In [120]:
import mysql.connector
import uuid
import time
import csv
import secrets
import json
from uuid import UUID
import numpy as np
from datetime import datetime

In [121]:
connection = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="root",
        database="UUIDTestDb")
cursor = connection.cursor()

In [122]:
_last_v7_timestamp = None

def uuid7(simulated_time_ns) -> UUID:
    global _last_v7_timestamp

    timestamp_ms = simulated_time_ns // 10**6
    if _last_v7_timestamp is not None and timestamp_ms <= _last_v7_timestamp:
        timestamp_ms = _last_v7_timestamp + 1
    _last_v7_timestamp = timestamp_ms
    uuid_int = (timestamp_ms & 0xFFFFFFFFFFFF) << 80
    uuid_int |= secrets.randbits(76)
    return UUID(int=uuid_int)

def generate_intervals(total, avg_interval):
    if avg_interval > 10**8:  # If average interval is greater than 100 seconds
        # Use exponential distribution for large intervals
        return np.random.exponential(avg_interval, total)
    else:
        # Use Poisson distribution for smaller intervals
        return np.random.poisson(avg_interval, total)

def generate_uuids(total, rpm):
    uuids = []
    # Calculate the average time between requests in nanoseconds
    avg_time_between_requests = (60 / rpm) * 10**9
    intervals = generate_intervals(total, avg_time_between_requests)
    start_time = time.time_ns()
    current_time = start_time

    for i in range(total):
        # Simulate the passage of time
        current_time += int(intervals[i])
        uuid = uuid7(current_time)
        uuids.append(str(uuid))

        # Print progress
        if (i + 1) % 100000 == 0:
            print(f"Generated {i + 1} UUIDs")

    return uuids

def clean_tables(cursor):
    tables = [
        'uuidv4',
        'uuidv7_100rpm',
        'uuidv7_1000rpm',
        'uuidv7_5000rpm',
        'uuidv7_10000rpm',
        'uuidv7_10000rpm_large1',
        'uuidv7_10000rpm_large2',
        'uuidv7_10000rpm_large3',
        'uuidv7_10000rpm_large4',
        'uuidv4_large1',
        'uuidv4_large2',
        'uuidv4_large3',
        'uuidv4_large4',
        'incremental',
        'tenant_request_v4',
        'tenant_request_mixed_100rpm',
        'tenant_request_mixed_1000rpm',
        'tenant_request_mixed_5000rpm',
        'tenant_request_mixed_10000rpm',
        'request_tenant_mixed_100rpm',
        'request_tenant_mixed_1000rpm',
        'request_tenant_mixed_5000rpm',
        'request_tenant_mixed_10000rpm'
    ]
    for table in tables:
        cursor.execute(f"TRUNCATE TABLE {table}")
    print("All tables cleaned.")


def get_table_size(cursor, table_name):
    cursor.execute(f"SELECT table_name, round(((data_length + index_length) / 1024), 2) as 'Size (KB)' FROM information_schema.TABLES WHERE table_schema = DATABASE() AND table_name = '{table_name}'")
    return cursor.fetchone()[1]


In [123]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv4 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_100rpm (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_1000rpm (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_5000rpm (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_10000rpm (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

# Additional tables with increasing row sizes for UUIDv7
cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_10000rpm_large1 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_10000rpm_large2 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_10000rpm_large3 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT,
    extra_blob BLOB
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv7_10000rpm_large4 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT,
    extra_blob BLOB,
    extra_json JSON
)
""")

# Additional tables with increasing row sizes for UUIDv4
cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv4_large1 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv4_large2 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv4_large3 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT,
    extra_blob BLOB
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS uuidv4_large4 (
    uuid CHAR(36) PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    extra_varchar VARCHAR(100),
    extra_text TEXT,
    extra_blob BLOB,
    extra_json JSON
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS incremental (
    id INT PRIMARY KEY,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tenant_request_v4 (
    tenant_id CHAR(36),
    request_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (tenant_id, request_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tenant_request_mixed_100rpm (
    tenant_id CHAR(36),
    request_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (tenant_id, request_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tenant_request_mixed_1000rpm (
    tenant_id CHAR(36),
    request_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (tenant_id, request_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tenant_request_mixed_5000rpm (
    tenant_id CHAR(36),
    request_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (tenant_id, request_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS tenant_request_mixed_10000rpm (
    tenant_id CHAR(36),
    request_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (tenant_id, request_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS request_tenant_mixed_100rpm (
    request_id CHAR(36),
    tenant_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (request_id, tenant_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS request_tenant_mixed_1000rpm (
    request_id CHAR(36),
    tenant_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (request_id, tenant_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS request_tenant_mixed_5000rpm (
    request_id CHAR(36),
    tenant_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (request_id, tenant_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS request_tenant_mixed_10000rpm (
    request_id CHAR(36),
    tenant_id CHAR(36),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (request_id, tenant_id)
)
""")

In [124]:
RECORDS_NUM = 1000

In [125]:
# Generate UUIDs in advance
print("Generating UUIDs...")

uuidv4_list = [str(uuid.uuid4()) for _ in range(RECORDS_NUM)]

# Generate UUIDv7 lists at different RPM rates
uuidv7_100rpm_list = generate_uuids(RECORDS_NUM, 100)
uuidv7_1000rpm_list = generate_uuids(RECORDS_NUM, 1000)
uuidv7_5000rpm_list = generate_uuids(RECORDS_NUM, 5000)
uuidv7_10000rpm_list = generate_uuids(RECORDS_NUM, 10000)

tenant_request_v4_list = [(uuidv4_list[i], str(uuid.uuid4())) for i in range(RECORDS_NUM)]

# Generate mixed lists at different RPM rates
tenant_request_mixed_100rpm_list = [(uuidv4_list[i], uuidv7_100rpm_list[i]) for i in range(RECORDS_NUM)]
tenant_request_mixed_1000rpm_list = [(uuidv4_list[i], uuidv7_1000rpm_list[i]) for i in range(RECORDS_NUM)]
tenant_request_mixed_5000rpm_list = [(uuidv4_list[i], uuidv7_5000rpm_list[i]) for i in range(RECORDS_NUM)]
tenant_request_mixed_10000rpm_list = [(uuidv4_list[i], uuidv7_10000rpm_list[i]) for i in range(RECORDS_NUM)]

print("UUID generation complete")

results = []

Generating UUIDs...
UUID generation complete


In [126]:
# Define dummy data
uuid_data = str(uuid.uuid4())
extra_varchar_data = 'a' * 100  # VARCHAR(100)
extra_text_data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit.' * 10  # TEXT
extra_blob_data = bytes([0] * 1024)  # BLOB with 1024 bytes
extra_json_data = json.dumps({"key": "value", "numbers": list(range(100))})  # JSON

In [116]:
# Test UUIDv4
print("Inserting UUIDv4...")
start_time = time.time()
for uuid_value in uuidv4_list:
    cursor.execute("INSERT INTO uuidv4 (uuid) VALUES (%s)", (uuid_value,))
connection.commit()
end_time = time.time()
uuidv4_time = end_time - start_time
uuidv4_size = get_table_size(cursor, "uuidv4")
results.append(('UUIDv4', uuidv4_time, uuidv4_size))

Inserting UUIDv4...


In [99]:
# Lists of UUIDv7 for different RPM rates
uuidv7_lists = {
    "100rpm": uuidv7_100rpm_list,
    "1000rpm": uuidv7_1000rpm_list,
    "5000rpm": uuidv7_5000rpm_list,
    "10000rpm": uuidv7_10000rpm_list
}

# Test UUIDv7 at different RPM rates
for rpm, uuidv7_list in uuidv7_lists.items():
    print(f"Inserting UUIDv7 at {rpm}...")
    start_time = time.time()
    for uuid_value in uuidv7_list:
        cursor.execute(f"INSERT INTO uuidv7_{rpm} (uuid) VALUES (%s)", (uuid_value,))
    connection.commit()
    end_time = time.time()
    insertion_time = end_time - start_time
    table_size = get_table_size(cursor, f"uuidv7_{rpm}")
    results.append((f'UUIDv7_{rpm}', insertion_time, table_size))

Inserting UUIDv7 at 100rpm...
Inserting UUIDv7 at 1000rpm...
Inserting UUIDv7 at 5000rpm...
Inserting UUIDv7 at 10000rpm...


In [100]:
# Test Tenant-Request V4
print("Inserting Tenant-Request V4...")
start_time = time.time()
for tenant_id, request_id in tenant_request_v4_list:
    cursor.execute("INSERT INTO tenant_request_v4 (tenant_id, request_id) VALUES (%s, %s)", (tenant_id, request_id))
connection.commit()
end_time = time.time()
tenant_request_v4_time = end_time - start_time
tenant_request_v4_size = get_table_size(cursor, "tenant_request_v4")
results.append(('Tenant-Request V4', tenant_request_v4_time, tenant_request_v4_size))

Inserting Tenant-Request V4...


In [101]:
# Lists of tenant-request mixed for different RPM rates
tenant_request_mixed_lists = {
    "100rpm": tenant_request_mixed_100rpm_list,
    "1000rpm": tenant_request_mixed_1000rpm_list,
    "5000rpm": tenant_request_mixed_5000rpm_list,
    "10000rpm": tenant_request_mixed_10000rpm_list
}

for rpm, tenant_request_mixed_list in tenant_request_mixed_lists.items():
    print(f"Inserting Tenant-Request Mixed at {rpm}...")
    start_time = time.time()
    for tenant_id, request_id in tenant_request_mixed_list:
        cursor.execute(f"INSERT INTO tenant_request_mixed_{rpm} (tenant_id, request_id) VALUES (%s, %s)", (tenant_id, request_id))
    connection.commit()
    end_time = time.time()
    insertion_time = end_time - start_time
    table_size = get_table_size(cursor, f"tenant_request_mixed_{rpm}")
    results.append((f'Tenant-Request Mixed_{rpm}', insertion_time, table_size))

Inserting Tenant-Request Mixed at 100rpm...
Inserting Tenant-Request Mixed at 1000rpm...
Inserting Tenant-Request Mixed at 5000rpm...
Inserting Tenant-Request Mixed at 10000rpm...


In [102]:
# Lists of request-tenant mixed for different RPM rates
request_tenant_mixed_lists = {
    "100rpm": tenant_request_mixed_100rpm_list,
    "1000rpm": tenant_request_mixed_1000rpm_list,
    "5000rpm": tenant_request_mixed_5000rpm_list,
    "10000rpm": tenant_request_mixed_10000rpm_list
}
# Test Request-Tenant Mixed at different RPM rates
for rpm, request_tenant_mixed_list in request_tenant_mixed_lists.items():
    print(f"Inserting Request-Tenant Mixed at {rpm}...")
    start_time = time.time()
    for tenant_id, request_id in request_tenant_mixed_list:
        cursor.execute(f"INSERT INTO request_tenant_mixed_{rpm} (request_id, tenant_id) VALUES (%s, %s)", (request_id, tenant_id))
    connection.commit()
    end_time = time.time()
    insertion_time = end_time - start_time
    table_size = get_table_size(cursor, f"request_tenant_mixed_{rpm}")
    results.append((f'Request-Tenant Mixed_{rpm}', insertion_time, table_size))

Inserting Request-Tenant Mixed at 100rpm...
Inserting Request-Tenant Mixed at 1000rpm...
Inserting Request-Tenant Mixed at 5000rpm...
Inserting Request-Tenant Mixed at 10000rpm...


In [127]:
# Test for UUIDv4 tables with increasing row sizes
uuidv4_tables = [
    ('uuidv4_large1', ('uuid', 'extra_varchar'), (uuidv4_list, extra_varchar_data)),
    ('uuidv4_large2', ('uuid', 'extra_varchar', 'extra_text'), (uuidv4_list, extra_varchar_data, extra_text_data)),
    ('uuidv4_large3', ('uuid', 'extra_varchar', 'extra_text', 'extra_blob'), (uuidv4_list, extra_varchar_data, extra_text_data, extra_blob_data)),
    ('uuidv4_large4', ('uuid', 'extra_varchar', 'extra_text', 'extra_blob', 'extra_json'), (uuidv4_list, extra_varchar_data, extra_text_data, extra_blob_data, extra_json_data))
]

for table_name, columns, data in uuidv4_tables:
    print(f"Inserting into {table_name}...")
    start_time = time.time()
    for i in range(RECORDS_NUM):
        cursor.execute(f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})", tuple(d[i] if isinstance(d, list) else d for d in data))
    connection.commit()
    end_time = time.time()
    insertion_time = end_time - start_time
    table_size = get_table_size(cursor, table_name)
    results.append((table_name, insertion_time, table_size))

Inserting into uuidv4_large1...
Inserting into uuidv4_large2...
Inserting into uuidv4_large3...
Inserting into uuidv4_large4...


In [128]:
# Test for UUIDv7 tables with increasing row sizes
uuidv7_tables = [
    ('uuidv7_10000rpm_large1', ('uuid', 'extra_varchar'), (uuidv7_10000rpm_list, extra_varchar_data)),
    ('uuidv7_10000rpm_large2', ('uuid', 'extra_varchar', 'extra_text'), (uuidv7_10000rpm_list, extra_varchar_data, extra_text_data)),
    ('uuidv7_10000rpm_large3', ('uuid', 'extra_varchar', 'extra_text', 'extra_blob'), (uuidv7_10000rpm_list, extra_varchar_data, extra_text_data, extra_blob_data)),
    ('uuidv7_10000rpm_large4', ('uuid', 'extra_varchar', 'extra_text', 'extra_blob', 'extra_json'), (uuidv7_10000rpm_list, extra_varchar_data, extra_text_data, extra_blob_data, extra_json_data))
]

for table_name, columns, data in uuidv7_tables:
    print(f"Inserting into {table_name}...")
    start_time = time.time()
    for i in range(RECORDS_NUM):
        cursor.execute(f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})", tuple(d[i] if isinstance(d, list) else d for d in data))
    connection.commit()
    end_time = time.time()
    insertion_time = end_time - start_time
    table_size = get_table_size(cursor, table_name)
    results.append((table_name, insertion_time, table_size))

Inserting into uuidv7_10000rpm_large1...
Inserting into uuidv7_10000rpm_large2...
Inserting into uuidv7_10000rpm_large3...
Inserting into uuidv7_10000rpm_large4...


In [129]:
# Print results
for result in results:
    print(f"{result[0]} insertion time: {result[1]:.2f} seconds")
    print(f"{result[0]} table size: {result[2]:.2f} KB")

uuidv4_large1 insertion time: 0.14 seconds
uuidv4_large1 table size: 16.00 KB
uuidv4_large2 insertion time: 0.23 seconds
uuidv4_large2 table size: 16.00 KB
uuidv4_large3 insertion time: 0.29 seconds
uuidv4_large3 table size: 16.00 KB
uuidv4_large4 insertion time: 0.25 seconds
uuidv4_large4 table size: 16.00 KB
uuidv7_10000rpm_large1 insertion time: 0.12 seconds
uuidv7_10000rpm_large1 table size: 16.00 KB
uuidv7_10000rpm_large2 insertion time: 0.21 seconds
uuidv7_10000rpm_large2 table size: 16.00 KB
uuidv7_10000rpm_large3 insertion time: 0.24 seconds
uuidv7_10000rpm_large3 table size: 16.00 KB
uuidv7_10000rpm_large4 insertion time: 0.27 seconds
uuidv7_10000rpm_large4 table size: 16.00 KB


In [105]:
# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"mysql_insertion_test_results_{RECORDS_NUM}_records_{timestamp}.csv"

with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['ID Type', 'Insertion Time (s)', 'Table Size (KB)'])
    csvwriter.writerows(results)

print(f"Results saved to {csv_filename}")


Results saved to mysql_insertion_test_results_100000_records_20240730_200437.csv


In [106]:
# Clean tables after the test
clean_tables(cursor)

All tables cleaned.


In [119]:
connection.close()